# **Banner Grabbing**

**Whats banner grabbing?**
- Sends a request to target
    - Get string value returned.
- Hopefully string contains information about target

In [ ]:
import argparse
import socket
import threading

def connection_scan(target_ip, target_port):
    """Attempts to create socket connection with target IP and port. If successful, the port is open, else, closed."""
    try:
        conn_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        conn_socket.connect((target_ip, target_port))

        conn_socket.send(b'Banner_query\r\n') # Banner Query.
        results = conn_socket.recv(100)

        print("[+] {}/tcp open".format(target_port))
        print("[+] {}".format(str(results))) # Print Banner
    except OSError:
        print("[-] {}/tcp closed".format(target_port))
    finally:
        conn_socket.close() # Ensure connection is closed.

def port_scan(target_ip,port_num):
    """Scan indicated ports for status. First, tries to resolve IP Address of provided hostnaem and enumerate ports."""

    try:
        target_ip = socket.gethostbyname(target_ip)
    except OSError:
        print("[^] Cannot resolve {}: Unknown host".format(target_ip))
        return # Exit scan if target IP not resolved.

    try:
        target_name = socket.gethostbyaddr(target_ip)
        print('[*] Scan Reusults for: {}'.format(target_name[0]))
    except OSError:
        print('[*] Scan Results for: {}'.format(target_ip))

    t = threading.Thread(target=connection_scan, args=(target_ip, int(port_num)))
    t.start()

def argument_parser():
    # CMD LINE Program helper
    """Allow user to specify target host and port."""
    
    parser = argparse.ArgumentParser(description="TCP Port Scanner. Accepts a hostname/IP address and list of ports to scan. Attempts to identify the service running on a port.")
    parser.add_argument("-o","--host",nargs="?",help="Host IP Address")
    parser.add_argument("-p","--ports",nargs="?",help="Comma-separated port list, such as '25,80,8080'")

    var_args = vars(parser.parse_args()) # Convert argument namespace to dictionary.
    return var_args

if __name__ == '__main__':
    try:
        user_args = argument_parser()
        host = user_args["host"]
        port_list = user_args["ports"].split(",") # Make a list from port list
        for port in port_list:
            port_scan(host,port)
    except AttributeError:
        print("Error. Please provide the command-line argument before running.")